In [1]:
import pandas as pd
import numpy as np
import os
import hashlib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:.5f}'.format

filepath = os.path.join(os.getcwd(),"Data")
labeled_dataset = pd.DataFrame.from_csv(os.path.join(filepath,"train.csv"), index_col = None)
"Dataset Size (Rows, Columns): " + str(labeled_dataset.shape)

'Dataset Size (Rows, Columns): (188318, 132)'

In [2]:
labeled_dataset.head(n=7) #mix of categorical and continuous variables

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.71837,0.33506,0.30260,0.67135,0.83510,0.56974,0.59465,0.82249,0.71484,2213.18000
1,2,A,B,A,A,A,A,A,A,B,...,0.43892,0.43659,0.60087,0.35127,0.43919,0.33831,0.36631,0.61143,0.30450,1283.60000
2,5,A,B,A,A,B,A,A,A,B,...,0.28965,0.31554,0.27320,0.26076,0.32446,0.38140,0.37342,0.19571,0.77443,3005.09000
3,10,B,B,A,B,A,A,A,A,B,...,0.44094,0.39113,0.31796,0.32128,0.44467,0.32792,0.32157,0.60508,0.60264,939.85000
4,11,A,B,A,B,A,A,A,A,B,...,0.17819,0.24741,0.24564,0.22089,0.21230,0.20469,0.20221,0.24601,0.43261,2763.85000
5,13,A,B,A,A,A,A,A,A,B,...,0.36446,0.40116,0.26847,0.46226,0.50556,0.36679,0.35925,0.34525,0.72679,5142.87000
6,14,A,A,A,A,B,A,A,A,A,...,0.38151,0.36377,0.24564,0.40455,0.47225,0.33483,0.35225,0.34224,0.38293,1132.22000


In [3]:
labeled_dataset.isnull().sum().sum() #no missing data!

0

In [4]:
cont_dataset = labeled_dataset.iloc[:,-15:].copy()
cont_dataset.head(n=7)

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,0.72630,0.24592,0.18758,0.78964,0.31006,0.71837,0.33506,0.30260,0.67135,0.83510,0.56974,0.59465,0.82249,0.71484,2213.18000
1,0.33051,0.73707,0.59268,0.61413,0.88583,0.43892,0.43659,0.60087,0.35127,0.43919,0.33831,0.36631,0.61143,0.30450,1283.60000
2,0.26184,0.35832,0.48420,0.23692,0.39707,0.28965,0.31554,0.27320,0.26076,0.32446,0.38140,0.37342,0.19571,0.77443,3005.09000
3,0.32159,0.55578,0.52799,0.37382,0.42227,0.44094,0.39113,0.31796,0.32128,0.44467,0.32792,0.32157,0.60508,0.60264,939.85000
4,0.27320,0.15999,0.52799,0.47320,0.70427,0.17819,0.24741,0.24564,0.22089,0.21230,0.20469,0.20221,0.24601,0.43261,2763.85000
5,0.54667,0.68176,0.63422,0.37382,0.30268,0.36446,0.40116,0.26847,0.46226,0.50556,0.36679,0.35925,0.34525,0.72679,5142.87000
6,0.47145,0.73707,0.61366,0.18914,0.29540,0.38151,0.36377,0.24564,0.40455,0.47225,0.33483,0.35225,0.34224,0.38293,1132.22000


In [5]:
cont_dataset.describe() #excluding loss, all the continuous variables are between 0 and 1-ish.

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000
mean,0.49386,0.50719,0.49892,0.49181,0.48743,0.49094,0.48497,0.48644,0.48551,0.49807,0.49351,0.49315,0.49314,0.49572,3037.33769
std,0.18764,0.20720,0.20210,0.21129,0.20903,0.20527,0.17845,0.19937,0.18166,0.18588,0.20974,0.20943,0.21278,0.22249,2904.08619
min,0.00002,0.00115,0.00263,0.17692,0.28114,0.01268,0.06950,0.23688,0.00008,0.00000,0.03532,0.03623,0.00023,0.17972,0.67000
25%,0.34609,0.35832,0.33696,0.32735,0.28114,0.33611,0.35018,0.31280,0.35897,0.36458,0.31096,0.31166,0.31576,0.29461,1204.46000
50%,0.47578,0.55578,0.52799,0.45289,0.42227,0.44094,0.43829,0.44106,0.44145,0.46119,0.45720,0.46229,0.36355,0.40740,2115.57000
75%,0.62391,0.68176,0.63422,0.65207,0.64331,0.65502,0.59104,0.62358,0.56682,0.61459,0.67892,0.67576,0.68997,0.72462,3864.04500
max,0.98498,0.86265,0.94425,0.95430,0.98367,0.99716,1.00000,0.98020,0.99540,0.99498,0.99874,0.99848,0.98849,0.84485,121012.25000


In [6]:
cont_dataset['logloss'] = np.log(cont_dataset['loss'])
cont_scaler = StandardScaler()
cont_dataset_scaled = pd.DataFrame(cont_scaler.fit_transform(cont_dataset),columns=cont_dataset.columns)
cont_dataset_scaled.describe()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss,logloss
count,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000,188318.00000
mean,-0.00000,-0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00000,0.00000,-0.00000,-0.00000,-0.00000,0.00000,0.00000
std,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
min,-2.63188,-2.44226,-2.45559,-1.49032,-0.98688,-2.32989,-2.32820,-1.25173,-2.67217,-2.67956,-2.18460,-2.18176,-2.31656,-1.42029,-1.04566,-9.95946
25%,-0.78753,-0.71848,-0.80135,-0.77835,-0.98688,-0.75431,-0.75537,-0.87093,-0.69656,-0.71814,-0.87038,-0.86660,-0.83364,-0.90390,-0.63114,-0.72855
50%,-0.09634,0.23452,0.14385,-0.18423,-0.31173,-0.24358,-0.26162,-0.22760,-0.24252,-0.19839,-0.17311,-0.14738,-0.60905,-0.39694,-0.31740,-0.03472
75%,0.69309,0.84253,0.66948,0.75848,0.74578,0.79931,0.59442,0.68788,0.44762,0.62689,0.88403,0.87195,0.92508,1.02885,0.28467,0.70726
max,2.61732,1.71556,2.20348,2.18885,2.37409,2.46608,2.88613,2.47662,2.80686,2.67336,2.40889,2.41294,2.32806,1.56922,40.62387,4.94956


In [7]:
cat_dataset = labeled_dataset.iloc[:,1:-15].copy()
cat_dataset.head(n=7) #categorical variables

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
0,A,B,A,B,A,A,A,A,B,A,...,J,G,BU,BC,C,AS,S,A,O,LB
1,A,B,A,A,A,A,A,A,B,B,...,K,K,BI,CQ,A,AV,BM,A,O,DP
2,A,B,A,A,B,A,A,A,B,B,...,F,A,AB,DK,A,C,AF,A,I,GK
3,B,B,A,B,A,A,A,A,B,A,...,K,K,BI,CS,C,N,AE,A,O,DJ
4,A,B,A,B,A,A,A,A,B,B,...,G,B,H,C,C,Y,BM,A,K,CK
5,A,B,A,A,A,A,A,A,B,A,...,F,B,BI,CS,A,AS,AE,A,K,DJ
6,A,A,A,A,B,A,A,A,A,A,...,F,B,BI,DK,A,J,AF,A,K,DJ


In [8]:
pd.DataFrame(cat_dataset.nunique(),columns=['Unique Categories']).head(n=7)
pd.DataFrame(cat_dataset.nunique(),columns=['Unique Categories']).tail(n=7)
"There are " + str(list(cat_dataset.nunique()).count(2)) + " binary variables."

,Unique Categories
cat1,2
cat2,2
cat3,2
cat4,2
cat5,2
cat6,2
cat7,2


,Unique Categories
cat110,131
cat111,16
cat112,51
cat113,61
cat114,19
cat115,23
cat116,326


'There are 72 binary variables.'

In [9]:
min_freq=[] #least common category occurences
min2_freq=[] #second least common category occurences
avg_freq=[] #average category occurence number
freq_std=[] #standard deviation in category occurence number
for column in cat_dataset.columns:
    min_freq.append(int(cat_dataset[column].value_counts().min()))
    min2_freq.append(int(cat_dataset[column].value_counts().nsmallest().iloc[1]))
    avg_freq.append(int(cat_dataset[column].value_counts().mean()))
    freq_std.append(int(cat_dataset[column].value_counts().std()))
extra_desc = pd.DataFrame([min_freq,min2_freq,avg_freq,freq_std],columns=cat_dataset.columns)
extra_desc = extra_desc.rename({0:"minfreq",1:"min2freq",2:"avgfreq",3:"freqstd"})
cat_dataset.describe().append(extra_desc)

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116
count,188318,188318,188318,188318,188318,188318,188318,188318,188318,188318,...,188318,188318,188318,188318,188318,188318,188318,188318,188318,188318
unique,2,2,2,2,2,2,2,2,2,2,...,20,11,84,131,16,51,61,19,23,326
top,A,A,A,A,A,A,A,A,A,A,...,F,B,BI,CL,A,E,BM,A,K,HK
freq,141550,106721,177993,128395,123737,131693,183744,177274,113122,160213,...,47310,65512,152918,25305,128395,25148,26191,131693,43866,21061
minfreq,46768,81597,10325,59923,64581,56625,4574,11044,75196,28105,...,2,213,1,1,2,30,1,1,1,1
min2freq,141550,106721,177993,128395,123737,131693,183744,177274,113122,160213,...,2,520,1,1,3,144,1,1,2,1
avgfreq,94159,94159,94159,94159,94159,94159,94159,94159,94159,94159,...,9415,17119,2241,1437,11769,3692,3087,9911,8187,577
freqstd,67020,17765,118559,48417,41829,53081,126692,117542,26817,93414,...,13310,20051,16814,4649,32247,5361,4986,29975,12032,2008


In [10]:
# discard high cardinality variables
threshold = 0.0025
for column in cat_dataset.columns:
    if cat_dataset[column].value_counts().min() < threshold*cat_dataset.shape[0]:
        cat_dataset = cat_dataset.drop([column], axis=1)
        
min_freq=[]
min2_freq=[]
avg_freq=[]
freq_std=[]
for column in cat_dataset.columns:
    min_freq.append(int(cat_dataset[column].value_counts().min()))
    min2_freq.append(int(cat_dataset[column].value_counts().nsmallest().iloc[1]))
    avg_freq.append(int(cat_dataset[column].value_counts().mean()))
    freq_std.append(int(cat_dataset[column].value_counts().std()))
extra_desc = pd.DataFrame([min_freq,min2_freq,avg_freq,freq_std],columns=cat_dataset.columns)
extra_desc = extra_desc.rename({0:"minfreq",1:"min2freq",2:"avgfreq",3:"freqstd"})
cat_dataset.describe().append(extra_desc)

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cat76,cat79,cat80,cat81,cat82,cat83,cat85,cat86,cat87,cat98
count,188318,188318,188318,188318,188318,188318,188318,188318,188318,188318,...,188318,188318,188318,188318,188318,188318,188318,188318,188318,188318
unique,2,2,2,2,2,2,2,2,2,2,...,3,4,4,4,4,4,4,4,4,5
top,A,A,A,A,A,A,A,A,A,A,...,A,B,D,D,B,B,B,B,B,A
freq,141550,106721,177993,128395,123737,131693,183744,177274,113122,160213,...,181347,152929,137505,154385,147536,141534,186005,103852,166992,105492
minfreq,46768,81597,10325,59923,64581,56625,4574,11044,75196,28105,...,788,1668,783,788,2655,4958,514,1589,788,542
min2freq,141550,106721,177993,128395,123737,131693,183744,177274,113122,160213,...,6183,7064,3492,9013,18805,15788,788,10290,8819,10242
avgfreq,94159,94159,94159,94159,94159,94159,94159,94159,94159,94159,...,62772,47079,47079,47079,47079,47079,47079,47079,47079,37663
freqstd,67020,17765,118559,48417,41829,53081,126692,117542,26817,93414,...,102723,71378,63823,72187,67416,63555,92617,49317,80075,42304


In [11]:
# cat_dataset_numerical = cat_dataset.apply(LabelEncoder().fit_transform)
cat_dataset_dummy = pd.get_dummies(data=cat_dataset,drop_first=True) #convert categorial variables to one-hot encoding
cat_dataset_dummy.head(n=7)

,cat1_B,cat2_B,cat3_B,cat4_B,cat5_B,cat6_B,cat7_B,cat8_B,cat9_B,cat10_B,...,cat86_B,cat86_C,cat86_D,cat87_B,cat87_C,cat87_D,cat98_B,cat98_C,cat98_D,cat98_E
0,0,1,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,1,0
2,0,1,0,0,1,0,0,0,1,1,...,1,0,0,1,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,1,0
4,0,1,0,1,0,0,0,0,1,1,...,1,0,0,0,1,0,0,0,0,0
5,0,1,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
